# Sympy Guided Tour and Tutorial 

For ME EN 537 (Robotics), written by John Morrell, August 21, 2021 (Tarnarmour@gmail.com). Modified slightly and moved to Jupyter notebook by Marc Killpack, Sept 6, 2021, and Sept 11, 2023. 

## Introduction

Sympy is a really powerful library for doing symbolic math in Python. Sympy can be used to solve equations,
do logical operations, define and find derivatives of functions, generate python or C code, and much, much 
more. We will be using Sympy for most of the coding homework in the class. We will only be using Sympy to do
fairly basic things, but this tutorial should still be useful for anyone new to Sympy or symbolic solvers in 
general.

As a disclaimer, I am no expert in Sympy. This tutorial is based on the things I learned or looked up while 
working on previous projects and the coding assignments for this class. There may be some errors or inefficient
solutions to things throughout, and you are welcome to email me if you find any. I'll try to keep things updated
and useful.

## Installation

The first step is to install sympy. **If you are using the VM provided to the class, sympy will already be installed
and ready to go.** If you are using your own machine, you can install sympy through the terminal using the pip
tool. Run the following in your terminal:

>pip install sympy

You may need to use pip3 instead of pip if you have multiple installations of python on your machine. Or the installation command may be different depending on your operating system. Feel free to look up more detailed instructions.

Now that sympy is installed you should be able to include it in any python script you write.

In [2]:
import sympy as sp

You might also consider importing some of the more common functions directly for convenience, for example:

In [3]:
from sympy import sin, cos, sqrt, pprint

Personally I prefer to always be able to specify explicitly which namespace I'm using for things like this.

## Sympy Symbols 

Sympy uses a few custom data types. The most basic of these is the sympy.Symbol object. A symbol is basically
a variable with an attached string used for printing. We create them using the symbols function:

In [4]:
a = sp.symbols('a')

Multiple symbols can be made simultaneously as well:

In [5]:
b, c, d = sp.symbols('b, c, d')

Symbols act like variables in the mathematical sense, not the programming sense. We don't store numbers in these
symbols, we use them to make algebraic expressions or functions. For example:

In [6]:
expr = 2*a + 3*b**2 + sp.sqrt(c)
print(expr)

2*a + 3*b**2 + sqrt(c)


**Don't try to assign numbers to things you previously defined as symbols!!!**
(e.g. a = 3)

Instead, if we want to plug in numbers to a symbolic expression, we use the **subs** function.

In [7]:
substituted_expr = expr.subs(a, 3)
print(substituted_expr)
substituted_expr = expr.subs(a, b**2)  # We can substitute other symbols like this too, not just numbers
print(substituted_expr)
substituted_expr = expr.subs([(a, 1), (b, 2), (c, 3)])  # We can substitute multiple things at once by passing a list of symbol-value pairs in
print(substituted_expr)

3*b**2 + sqrt(c) + 6
5*b**2 + sqrt(c)
sqrt(3) + 14


You should see that the last print gave us "sqrt(3) + 14"; sympy evaluated the expression but did not return an 
approximate decimal answer, which we might need. To get this we use the .evalf() method or the N() function;

In [8]:
evaluated_expr = substituted_expr.evalf()
print(evaluated_expr)
print(sp.N(substituted_expr))

15.7320508075689
15.7320508075689


You can pass a precision to the evalf and N functions if you don't want to have 15 decimals displayed every time 
you print like this:

In [9]:
print(sp.N(evaluated_expr, 4))  # prints with 4 decimals of precision

15.73


It is important to remember that sympy symbols will work with just about any python function or operation; if a class 
or function does not have a defined rule for how to treat sympy variables it will break.

For example, the following line would throw an error when running, because numpy does not have a definition for how to take the sin of a sympy symbol (numpy is a linear algebra library in Python that is very useful, but will likely be unnecessary in this class due to our use of Sympy instead):
> import numpy as np

> np.sin(a)

If you want to take the sin of a symbol, use the sympy sin function instead. This is also true for things like sqrt, absolute values, etc.

In [10]:
print(sp.sin(a))

sin(a)


Symbols will be useful in this class because we will be defining the position of robot arms in terms of joint angles.
If we use symbols as the joint angles, we can see symbolic solutions for the robot arm position just by passing the 
symbols in the rest of our kinematic equations.

## Matrices

A lot of what we do in the class will involve rotation matrices and 4x4 homogeneous transformation matrices. We will
use the sympy Matrix object to represent these objects. If you have used numpy before, sympy Matrices behave in 
very similar ways.

In [11]:
M = sp.Matrix([[a, b], [c, 5]])
print(M)

Matrix([[a, b], [c, 5]])


Printing these matrices can be a bit difficult to read, especially as they get larger. Sympy has a very capable and
versatile printing tool. We will be using it to print more complicated expressions and matrices, though it's not
really necessary for anything in the class. It will make your life easier if you use it, though. To use it, just call
the sympy pprint (short for pretty_print) function:

In [12]:
sp.pprint(M)  # This should make reading matrices much easier

⎡a  b⎤
⎢    ⎥
⎣c  5⎦


We can multiply sympy matrices together using the @ operator, much like how numpy works (or MATLAB for that matter if you aren't familiar with numpy)

In [13]:
D = sp.Matrix([[1, 2], [3, 4]])
sp.pprint(D @ M)

⎡ a + 2⋅c    b + 10 ⎤
⎢                   ⎥
⎣3⋅a + 4⋅c  3⋅b + 20⎦


We can access specific elements or sections of matrices using the standard python slicing rules (which is very similar to indexing in MATLAB to access a specific matrix or array value). If you're not familiar with these, you can look them up as there are many good explanations for them online. Sympy matrices work the same as numpy arrays in this aspect. However, the principle is simple enough that you can likely figure it out by looking at examples below and in the homework. 

In [14]:
part = M[0:2, 1]
sp.pprint(part)

⎡b⎤
⎢ ⎥
⎣5⎦


We can still use subs and evalf just like before here as well

In [15]:
subs_M = M.subs([(a, 1), (b, -3), (c, sp.pi)])
sp.pprint(subs_M)
sp.pprint(sp.N(subs_M))

⎡1  -3⎤
⎢     ⎥
⎣π  5 ⎦
⎡      1.0         -3.0⎤
⎢                      ⎥
⎣3.14159265358979  5.0 ⎦


## Solving Equations

One of the most useful but simple things to use sympy for is solving equations. To do so, we use the solve function.
Simply pass in an expression and a symbol to solve for. The solve function assumes that the expression is set equal
to 0, so to solve 2*a = 5 for a, we would rearrange like so:

In [16]:
expr = 2 * a - 5
sol = sp.solve(expr, a)  # to use solve, pass in an expression or expressions and a symbol or symbols to solve for
print(sol)

[5/2]


There are a lot of problems that can show up when solving things. Some are fairly obvious; if an expression has no
solution, solve will return an empty solution:

In [17]:
sol = sp.solve(sp.sqrt(a) + 5, a)
print(sol)

[]


Some can be more complicated. In the expression below, we get no solution because the solver has assumed that b is 
positive, which would lead to no solution.

In [18]:
sol = sp.solve(sp.sqrt(a) + b, a)
print(sol)

[]


When we define symbols, we can give them assumptions like positive, negative, real, imaginary, etc. This can help
the solver succeed, especially on more complicated expressions.

In [19]:
b = sp.Symbol('b', positive=False)
sol = sp.solve(sp.sqrt(a) + b, a)
print(sol)

expr = [5*a + 4*b + 3*c, 3*b - c]  # here I'm using a list of expressions instead of a single expression
sol = sp.solve(expr, a)  # if you don't give a symbol to solve for, sympy will try to solve for all symbols
print(sol)  # sol is a dictionary of solutions for a and b

[b**2]
{a: -4*b/5 - 3*c/5}


## More Advanced Printing

If you have installed the required rendering tools (in extensions, it's called "Jupyter Notebook Renderers"), we can also use latex to print any complicated expressions to the screen (or even to file). Below is an example of this. This is a complex example of calculating kinetic and potential energy, and then coming up with the equations of motion. It is one of the main case studies in the controls class, but is also just useful to show how we can disply or render complicated symbolic expressions, or export them as latex code for a report or paper.  

In [1]:
from IPython.display import Math, display
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector.printing import vpprint, vlatex

# defining all necessary mathematical variables as sympy "symbols" so that we can use them accordingly
t, m, ell, g = sp.symbols('t, m, ell, g')

# define theta and theta_dot as functions of time so we can take the time derivative
theta = dynamicsymbols('theta')
theta_dot = theta.diff(t)

#calculate the kinetic energy
K = 1/2.0*m*ell**2/3*theta_dot**2

#calculate the potential energy
P = m*g*ell/2.0*sp.sin(theta)

#calculate the lagrangian
L = K-P

#calculate the Euler-Lagrange equations of motion for theta variable - this is something you have not
# learned how to do yet in 537, but would learn at the start of the controls class. 
EL_case_studyA_v2 = sp.simplify((L.diff(theta_dot)).diff(t) - L.diff(theta))


# this is the part we probably care about (since it's showing how we can print the resulting expressions)
print("\n\n\n\n Euler-Lagrange equations for case study A:")
display(EL_case_studyA_v2)

# to see the dot notation:
display(Math(vlatex(EL_case_studyA_v2)))

# if you want dot notation (instead of d/dt) in your latex code, this will work too:
vlatex(EL_case_studyA_v2)





 Euler-Lagrange equations for case study A:


ell*m*(0.333333333333333*ell*Derivative(theta(t), (t, 2)) + 0.5*g*cos(theta(t)))

<IPython.core.display.Math object>

'\\ell m \\left(0.333333333333333 \\ell \\ddot{\\theta} + 0.5 g \\cos{\\left(\\theta \\right)}\\right)'